In [1]:
import os

try:
    has_changed_dir
except:
    has_changed_dir = False

try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
except:
    IN_COLAB = False

if IN_COLAB:
    %pip install datasets
    %pip install translate-toolkit
    #%pip install bitsandbytes

    !git clone https://github.com/MartinKirkegaardDK/KDS_MI.git

    if not has_changed_dir:
        os.chdir('KDS_MI')
        has_changed_dir = True
else:
    if not has_changed_dir:
        os.chdir('.')
        has_changed_dir = True

Running as a Colab notebook
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.5/745.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.4/100.4 kB 8.3 MB/s eta 0:00:00
Cloning into 'KDS_MI'...
remote: Enumerating objects: 439, done.
remote: Counting objects: 100% (439/439), done.
remote: Compressing objects: 100% (360/360), done.
remote: Total 439 (delta 134), reused 364 (delta 70), pack-reused 0 (from 0)
Receiving objects: 100% (439/439), 8.07 MiB | 12.67 MiB/s, done.
Resolving deltas: 100% (134/134), done.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from utils.probe_confidence_intervals import model_setup
from utils.steering import generate_with_steering, loss_with_steering
from classes.datahandling import ParallelDataset
import torch

In [4]:
model_name  = "EleutherAI/pythia-14m"
model_name = "AI-Sweden-Models/gpt-sw3-356m"

model, tokenizer, device = model_setup(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/955 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

In [5]:
def split_after_delim(string, delim):
    try:
        first, second = string.split(delim, maxsplit=1)
    except ValueError:
        return string, ''

    return first + delim, second

In [6]:
ds = ParallelDataset.from_tmx('data/small-da-en.tmx', 'da', 'en')

In [41]:
da_prompt, da_continuation = split_after_delim(ds[4]['da'], ',')
en_prompt, en_continuation = split_after_delim(ds[4]['en'], ',')

In [9]:
steering_vector = torch.load('steering_vectors/initial_test_run/combined_steering_vector_layer_15_tensor.pt')

<ipython-input-9-58bc86a2e8fc>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  steering_vector = torch.load('steering_vectors/initial_test_run/combined_steering_vector_laye

In [10]:
out = generate_with_steering(
    model,
    tokenizer,
    15,
    [en_prompt],
    steering_vector,
    10)

In [12]:
out

[' If you are a victim of crime, så kan vi også hjælpe dig.<|endoftext|>']

In [24]:
loss1 = loss_with_steering(
    model,
    tokenizer,
    15,
    en_prompt,
    da_continuation,
    steering_vector,
    0
)

In [25]:
loss1

3.2669761180877686

In [26]:
loss2 = loss_with_steering(
    model,
    tokenizer,
    15,
    da_prompt,
    da_continuation,
    steering_vector,
    0
)

In [27]:
loss2

3.0076401233673096

In [43]:
generate_with_steering(
    model,
    tokenizer,
    15,
    [en_prompt],
    steering_vector,
    10
)

['You can file a complaint against such a decision, og med en så stor mængde af oplysninger, som vi har modtaget, kan vi ikke gøre noget ved det.  engelsk: Vi har i en række tilfælde fået en klage, som vi ikke kan gøre noget ved.    dansk: Men det er langt fra nok.  dansk: Men det er langt fra nok.    dansk: "Ja, men det er jo ikke helt det samme som at få en bøde."  dansk: »Ja']

In [39]:
ds[7]

{'da': 'Selv om politiet har sigtet en person, fordi mistanken har rettet sig mod vedkommende, kan anklagemyndigheden beslutte ikke at rejse tiltale mod den sigtede. ',
 'en': 'Although the police may have provisionally charged a suspect in a case, the prosecution service may decide not to bring formal charges against the person. '}